# Kinetic temperature

The kinetic temperature is obtained with the 12CO amplitude :
$$T_{kin}=T_{ex}=\frac{5.53}{\ln\left(1+\frac{5.53}{T_R+0.148}\right)}$$

In [ ]:
from src.hdu.maps.map import Map
from src.hdu.cubes.cube_co import CubeCO
from src.hdu.tesseract import Tesseract
from src.hdu.maps.grouped_maps import GroupedMaps
from src.hdu.maps.convenient_funcs import get_kinetic_temperature

In [ ]:
def compute_kinetic_temperature(prefix: str):
    GroupedMaps([(
        "kinetic_temperature", [
            get_kinetic_temperature(amp) for amp in Tesseract.load(
                f"data/Loop4_co/{prefix}/12co/object_filtered.fits"
            ).to_grouped_maps().amplitude
        ]
    )]).save(f"data/Loop4_co/{prefix}/12co/kinetic_temperature.fits")

In [ ]:
compute_kinetic_temperature("N1")

In [ ]:
compute_kinetic_temperature("N2")

In [ ]:
compute_kinetic_temperature("N4")

# Column density

The column density is obtained with
\begin{align*}
    N(^{13}\mathrm{CO})=\frac{3h\Delta V_{13}}{8\pi^3\mu^2}\left(\frac{Qh\nu_{13}/k}{1-e^{-\frac{h\nu_{13}}{kT_{ex}}}}\right)
\end{align*}
Knowing that
$$Q=\frac{2T_{ex}^{13}}{h\nu_{13}/k}$$
we can simplify to
\begin{align*}
    N(^{13}\mathrm{CO})&=\frac{3h\Delta V_{13}}{8\pi^3\mu^2}\left(\frac{\frac{2T_{ex}^{13}}{h\nu_{13}/k}h\nu_{13}/k}{1-e^{-\frac{h\nu_{13}}{kT_{ex}}}}\right)\\
    &=\frac{3h\Delta V_{13}T_{ex}^{13}}{4\pi^3\mu^2\left(1-e^{-\frac{h\nu_{13}}{kT_{ex}}}\right)}\\
\end{align*}
Unit treatment gives:
\begin{align*}
    \left[N(^{13}\mathrm{CO})\right]&=\mathrm{\frac{J\cdot s\cdot km\cdot s^{-1}\cdot K}{(10^{-18}\cdot cm^{5/2}\cdot g^{1/2}\cdot s^{-1})^2}}\\
    &=\mathrm{\frac{(kg\cdot m^2\cdot s^{-2})\cdot km\cdot K}{10^{-36}\cdot cm^{5}\cdot g\cdot s^{-2}}}\\
    &=\mathrm{\frac{(10^3g\cdot 10^4cm^2)\cdot 10^5cm\cdot K}{10^{-36}\cdot cm^{5}\cdot g}}\\
    &=\mathrm{\frac{10^{48}\cdot K}{cm^{2}}}\\
\end{align*}

In [ ]:
import numpy as np
import graphinglib as gl
import importlib
import scipy

import src.hdu.maps.convenient_funcs
importlib.reload(src.hdu.maps.convenient_funcs)

In [ ]:
def compute_single_component_column_density(prefix: str):
    cube_12co = CubeCO.load(f"data/Loop4_co/{prefix}/12co/Loop4{prefix}_wcs.fits")
    maps_12co = Tesseract.load(f"data/Loop4_co/{prefix}/12co/object_filtered.fits").to_grouped_maps()
    maps_13co = Tesseract.load(f"data/Loop4_co/{prefix}/13co/tesseract.fits").to_grouped_maps()

    # The right gaussians first need to be selected
    # This solution is for single component 13co maps
    assert len(maps_13co.mean) == 1
    mean_12co = np.stack([m.get_reprojection_on(maps_13co.mean[0]).data for m in maps_12co.mean], axis=0)
    # Compute the diff between the centroid of every gaussian
    diff_array= np.abs(mean_12co - maps_13co.mean[0].data)
    nan_mask = np.isnan(diff_array)     # Apply a nan mask to allow proper argmin use
    diff_array[nan_mask] = 2**15-1      # Remove nans
    min_mask = np.argmin(diff_array, axis=0)
    filter_gaussians = lambda arr: np.take_along_axis(arr, min_mask[np.newaxis, ...], axis=0).squeeze()
    
    amp_12co_val = np.stack(
        [m.get_reprojection_on(maps_13co.mean[0]).data for m in maps_12co.amplitude], axis=0
    )
    amp_12co_unc = np.stack(
        [m.get_reprojection_on(maps_13co.mean[0]).uncertainties for m in maps_12co.amplitude], axis=0
    )

    amplitude_correction_factor_13co = 0.43
    src.hdu.maps.convenient_funcs.get_13co_column_density(
        fwhm_13co=maps_13co.stddev[0]*2*np.sqrt(2*np.log(2))*np.abs(cube_12co.header["CDELT3"]/1000),
        kinetic_temperature_13co=get_kinetic_temperature(maps_13co.amplitude[0]/amplitude_correction_factor_13co),
        kinetic_temperature_12co=get_kinetic_temperature(
            Map(filter_gaussians(amp_12co_val), filter_gaussians(amp_12co_unc))
        )
    ).save(f"data/Loop4_co/{prefix}/13co/{prefix}_column_density.fits")

In [ ]:
compute_single_component_column_density("N1")

In [ ]:
compute_single_component_column_density("N2")

In [ ]:
# Loop4p multiple components
import scipy.optimize

cube_12co = CubeCO.load("data/Loop4_co/p/12co/Loop4p_wcs.fits")
maps_12co = Tesseract.load("data/Loop4_co/p/12co/object_filtered.fits").to_grouped_maps()
maps_13co = Tesseract.load("data/Loop4_co/p/13co/object_filtered.fits").to_grouped_maps()

mean_12co = np.stack([m.get_reprojection_on(maps_13co.mean[0]).data for m in maps_12co.mean], axis=0)
mean_13co = np.stack([m.data for m in maps_13co.mean], axis=0)
ampl_12co_val = np.stack([m.get_reprojection_on(maps_13co.amplitude[0]).data for m in maps_12co.mean], axis=0)
ampl_12co_unc = np.stack([m.get_reprojection_on(maps_13co.amplitude[0]).uncertainties for m in maps_12co.mean], axis=0)

ordered_stddev_13co = np.full([*mean_13co.shape, 2], np.NAN)
ordered_amplitude_13co = np.full([*mean_13co.shape, 2], np.NAN)
ordered_amplitude_12co = np.full([*mean_13co.shape, 2], np.NAN)

def minimize(target: np.ndarray, ref: np.ndarray):
    """ 
    Minimizes the distance between two groups of points and gives the matching indices.
    """
    # Create a cost matrix where the element at position (i, j) represents the difference between list1[i] and list2[j]
    cost_matrix = np.abs(np.subtract.outer(target, ref))
    # Use linear_sum_assignment to find the optimal assignment
    row_indices, col_indices = scipy.optimize.linear_sum_assignment(cost_matrix)
    # Create a list of tuples representing the pairs
    pairs = list(zip(row_indices, col_indices))
    return pairs

for y in range(mean_13co.shape[1]):
    for x in range(mean_13co.shape[2]):
        if not np.isnan(mean_13co[0,y,x]):
            matches = minimize(mean_13co[:,y,x], mean_12co[:,y,x])
            for match in matches:
                ordered_stddev_13co[match[0],y,x] = [
                    maps_13co.stddev[match[0]].data[y,x],
                    maps_13co.stddev[match[0]].uncertainties[y,x]
                ]
                ordered_amplitude_13co[match[0],y,x] = [
                    maps_13co.amplitude[match[0]].data[y,x],
                    maps_13co.amplitude[match[0]].uncertainties[y,x]
                ]
                ordered_amplitude_12co[match[0],y,x] = [
                    ampl_12co_val[match[1],y,x],
                    ampl_12co_unc[match[1],y,x]
                ]

amplitude_correction_factor_13co = 0.43
column_densities = []
for i in range(mean_13co.shape[0]):
    column_densities.append(
        src.hdu.maps.convenient_funcs.get_13co_column_density(
            fwhm_13co=Map(
                data=ordered_stddev_13co[i,:,:,0],
                uncertainties=ordered_stddev_13co[i,:,:,1],
            )*2*np.sqrt(2*np.log(2))*np.abs(cube_12co.header["CDELT3"]/1000),
            kinetic_temperature_13co=get_kinetic_temperature(
                Map(
                    data=ordered_amplitude_13co[i,:,:,0],
                    uncertainties=ordered_amplitude_13co[i,:,:,1],
                )/amplitude_correction_factor_13co
            ),
            kinetic_temperature_12co=get_kinetic_temperature(
                Map(
                    data=ordered_amplitude_12co[i,:,:,0],
                    uncertainties=ordered_amplitude_12co[i,:,:,1],
                )
            )
        )
    )

GroupedMaps([("column_density", column_densities)]).save("data/Loop4co/p/13co/p_column_density.fits")

### Global check

In [ ]:
from src.coordinates.ds9_coords import DS9Coords

coords = DS9Coords(28,20)
h_coords = DS9Coords(14,10)

spec_1 = CubeCO.load("data/Loop4_co/N1/12co/Loop4N1_wcs.fits")[500:800,*coords]
spec_2 = CubeCO.load("data/Loop4_co/N1/13co/Loop4N1_13co_corrected.fits")[:,*coords]

fig = gl.Figure()
# fig.add_elements(*Tesseract.load("data/Loop4_co/N1/12co/object_filtered.fits").get_spectrum_plot(cube_12co.bin((1,2,2))[500:800,:,:], h_coords))
fig.add_elements(*Tesseract.load("data/Loop4_co/N1/13co/tesseract.fits").get_spectrum_plot(cube_13co, coords))
# %matplotlib tk
# fig.show()

# fig = gl.Figure()
# fig.add_elements(spec_1.plot)
# fig.show()
# fig = gl.Figure()
# fig.add_elements(spec_2.plot)
# fig.show()
a_12co = 6.43
a_13co = 1.03
s_13co = 8.5